In [ ]:
import SimpleITK as sitk

def resample_image_itk(imagePath,preferenceNii, newSpacing=[1.0,1.0,2.0], resamplemethod=sitk.sitkNearestNeighbor): 
    image = sitk.ReadImage(preferenceNii)

    resample = sitk.ResampleImageFilter()

    resample.SetInterpolator(resamplemethod) 
    resample.SetOutputDirection(image.GetDirection()) 
    resample.SetOutputOrigin(image.GetOrigin()) 

    newSpacing = np.array(newSpacing, float)
    newSize = image.GetSize() / newSpacing * image.GetSpacing()
    newSize = np.around(newSize,decimals=0)
    newSize = newSize.astype(np.int)

    resample.SetSize(newSize.tolist()) 
    resample.SetOutputSpacing(newSpacing) 
    image_resample = sitk.ReadImage(imagePath)
    newimage = resample.Execute(image_resample)
    return newimage 
    
def expand_dilation(mask_img_arr,iterations = 5):
    shape_nrrd = mask_img_arr.shape
    mask_img_arr_expand = np.zeros(shape_nrrd)
    for index in range(shape_nrrd[0]):
        mask_img_arr_expand[index,:,:] = scipy.ndimage.binary_dilation(mask_img_arr[index,:,:],iterations = iterations).astype('uint16')
    return mask_img_arr_expand


def diffROI(mask_img_arr,mask_img_arr_expand):

    mask_img_arr[mask_img_arr != 0] = 1
    mask_img_arr_expand[mask_img_arr_expand != 0] = 1

    shape_nrrd = mask_img_arr.shape
    mask_img_arr_border = np.zeros(shape_nrrd)   
    for index in range(shape_nrrd[0]):
        for x in range(shape_nrrd[1]):
            for y in range(shape_nrrd[2]):
                if mask_img_arr[index,x,y] != mask_img_arr_expand[index,x,y]:
                    mask_img_arr_border[index,x,y] = 1
    return mask_img_arr_border
    

def array2nii(image_array, out_path, NIIimage_resample):

    image2 = NIIimage_resample
    
    origin = image2.GetOrigin()  
    spacing = image2.GetSpacing()  
    direction = image2.GetDirection()  
    
    image3 = sitk.GetImageFromArray(image_array)
    image3.SetSpacing(spacing)
    image3.SetDirection(direction)
    image3.SetOrigin(origin)
    sitk.WriteImage(image3, out_path)

In [ ]:
import os
import numpy as np
import SimpleITK as sitk


img_folder = r''
roi_folder = r''


img1_folder = r''
roi1_folder = r''


os.makedirs(img1_folder, exist_ok=True)
os.makedirs(roi1_folder, exist_ok=True)


newSpacing=[1.0,1.0,2.0]


for filename in os.listdir(img_folder):
    if filename.endswith('.nii.gz'):
        
        img_path = os.path.join(img_folder, filename)
        roi_path = os.path.join(roi_folder, filename.replace('image', 'mask'))
        
        
        Niimage = sitk.ReadImage(img_path)
        mask_img = sitk.ReadImage(roi_path)
        
        
        Niimage_resample = resample_image_itk(img_path,preferenceNii=img_path,newSpacing = newSpacing, resamplemethod=sitk.sitkNearestNeighbor)
        mask_img_resample = resample_image_itk(roi_path,preferenceNii=img_path,newSpacing = newSpacing, resamplemethod=sitk.sitkNearestNeighbor)
        
        img1_path = os.path.join(img1_folder, filename.replace('image', 'img1'))
        roi1_path = os.path.join(roi1_folder, filename.replace('image', 'roi1'))
        
        array2nii(sitk.GetArrayFromImage(Niimage_resample), img1_path, Niimage_resample)
        array2nii(sitk.GetArrayFromImage(mask_img_resample), roi1_path, mask_img_resample)